In [1]:
import xml.etree.ElementTree as ET

In [2]:
rml_file_path = './data/psg_audio/rml_clean/00000995-100507.rml'

In [3]:
tree = ET.parse(rml_file_path)
root = tree.getroot()

In [4]:
print(root.tag, root.attrib.get('Version'))

{http://www.respironics.com/PatientStudy.xsd}PatientStudy 6


In [5]:
taglists = []

for child in root:
    taglists.append(child.tag.split('}')[1]) 

taglists

['Patient',
 'ChannelConfig',
 'Acquisition',
 'CustomEventTypeDefs',
 'AcquisitionCommentDefs',
 'ScoringData',
 'AnalysisOptions',
 'ReportCalcOptions',
 'EventFilters',
 'TrendChannelMappings',
 'BodyPositionState']

In [6]:
for child in root[1]:
    print(child.tag, child.text)

{http://www.respironics.com/PatientStudy.xsd}ConfigName LDxS_PSG_AASM-C3-C4
{http://www.respironics.com/PatientStudy.xsd}Channels 
         


In [7]:
ns = { 'study' : 'http://www.respironics.com/PatientStudy.xsd'}

for patient in root.findall('study:Patient', ns):
    gender = patient.find('study:Gender', ns)
    print(gender.text)

Female


In [8]:
for AcqNumber in root.findall('study:Acquisition', ns):
    number = AcqNumber.find('study:AcqNumber', ns)
    print(number.text)

995


In [9]:
ns = { 'study' : 'http://www.respironics.com/PatientStudy.xsd'}

acq = root.findall('study:Acquisition', ns)

acq_number = [x.find('study:AcqNumber', ns).text for x in acq]

print(acq_number)

['995']


In [10]:
root.find('study:Acquisition', ns).find('study:Sessions', ns).find('study:Session', ns).find('study:Segments', ns)

<Element '{http://www.respironics.com/PatientStudy.xsd}Segments' at 0x00000165D6E17590>

In [11]:
segment = root.find('study:Acquisition', ns).find('study:Sessions', ns).find('study:Session', ns).find('study:Segments', ns)

duration = [x.find('study:Duration', ns).text for x in segment]

print(duration)

['3600', '3600', '3600', '3600', '3491']


In [34]:
stages = root.find('study:ScoringData', ns).find('study:StagingData', ns).find('study:UserStaging', ns).find('study:NeuroAdultAASMStaging', ns).findall('study:Stage', ns)

all_sleep = [x.attrib for x in stages]

for i in range(len(all_sleep)):
    if i < len(all_sleep) - 1 :
        all_sleep[i]['End'] = int(all_sleep[i + 1]['Start'])
    else:
        all_sleep[i]['End'] = int(all_sleep[i]['Start'])

wake = [x for x in all_sleep if x.get('Type') == 'Wake']

rem = [x for x in all_sleep if x.get('Type') == 'REM']


In [ ]:
all_sleep

In [13]:
events = root.find('study:ScoringData', ns).find('study:Events', ns).findall('study:Event', ns)

respiratory = [x.attrib for x in events if x.attrib.get('Family') == 'Respiratory']

for i in range(len(respiratory)):
    respiratory[i]['End'] = float(respiratory[i]['Start']) + float(respiratory[i]['Duration'])

In [14]:
respiratory[0]

{'Family': 'Respiratory',
 'Type': 'Hypopnea',
 'Start': '3752.5',
 'Duration': '10'}

In [15]:
# respiratory[0]['End'] = float(respiratory[0]['Start']) + float(respiratory[0]['Duration'])

In [16]:
for i in range(len(respiratory)):
    respiratory[i]['End'] = float(respiratory[i]['Start']) + float(respiratory[i]['Duration'])

In [18]:
respiratory[0]

{'Family': 'Respiratory',
 'Type': 'Hypopnea',
 'Start': '3752.5',
 'Duration': '10',
 'End': 3762.5}

In [57]:
type(wake[0]['Start'])

str

In [37]:
import pandas as pd

In [73]:
# wake_df = pd.DataFrame.from_dict(wake)
# rem_df = pd.DataFrame.from_dict(rem)

# stages_df = pd.concat([wake_df, rem_df], axis=0).astype({'Start' : 'int' , 'End' : 'int'}).sort_values(by='Start').reset_index(drop=True)

# stages_df['patient_id'] = 0
# patient_id = {0 : acq_number}

# stages_df['patient_id'] = stages_df['patient_id'].replace(patient_id)

# breath_df = pd.DataFrame.from_dict(respiratory).astype({'Start' : 'float' , 'End' : 'float'}).sort_values(by='Start').reset_index(drop=True).drop('Family', axis=1)

In [147]:
stages_df = pd.DataFrame.from_dict(all_sleep).astype({'Start' : 'int' , 'End' : 'int'}).sort_values(by='Start').reset_index(drop=True)

stages_df['patient_id'] = 0
patient_id = {0 : acq_number}

stages_df['patient_id'] = stages_df['patient_id'].replace(patient_id)

breath_df = pd.DataFrame.from_dict(respiratory).astype({'Start' : 'float' , 'End' : 'float', 'Duration' : 'float'}).sort_values(by='Start').reset_index(drop=True).drop('Family', axis=1)

In [148]:
stages_df

,Type,Start,End,patient_id
0,Wake,0,210,995
1,NonREM1,210,300,995
2,NonREM2,300,870,995
3,Wake,870,930,995
4,NonREM1,930,960,995
5,NonREM2,960,1920,995
6,Wake,1920,1980,995
7,NonREM1,1980,2010,995
8,NonREM2,2010,3300,995
9,NonREM1,3300,3330,995


In [150]:
breath_df

,Type,Start,Duration,End
0,Hypopnea,3752.5,10.0,3762.5
1,Hypopnea,3783.0,12.0,3795.0
2,Hypopnea,3813.5,10.5,3824.0
3,Hypopnea,3842.0,10.0,3852.0
4,Hypopnea,3878.0,11.0,3889.0
...,...,...,...,...
206,ObstructiveApnea,17610.0,10.0,17620.0
207,ObstructiveApnea,17642.5,16.0,17658.5
208,ObstructiveApnea,17682.0,18.5,17700.5
209,Hypopnea,17820.5,11.5,17832.0


In [151]:
def if_start_end(values, df):
    for i in range(len(df)):
        if df[['Start', 'End']].loc[i][0] <= values <= df[['Start', 'End']].loc[i][1]:
            pass
        else:
            return values == 'no'

In [157]:
breath_df.loc[breath_df['Type'] == 'Hypopnea'].loc[breath_df['Duration'] == 18]

,Type,Start,Duration,End
71,Hypopnea,9104.5,18.0,9122.5


In [158]:
breath_df['Type'].value_counts()

Hypopnea            131
ObstructiveApnea     67
MixedApnea           13
Name: Type, dtype: int64

In [161]:
print(breath_df.loc[breath_df['Type'] == 'ObstructiveApnea'].Duration.max(),
breath_df.loc[breath_df['Type'] == 'MixedApnea'].Duration.max())

21.0 17.5


In [163]:
breath_df.loc[breath_df['Type'] == 'Hypopnea'].loc[breath_df['Duration'] == 18]


,Type,Start,Duration,End
71,Hypopnea,9104.5,18.0,9122.5


In [164]:
breath_df.loc[breath_df['Type'] == 'ObstructiveApnea'].loc[breath_df['Duration'] == 21]

,Type,Start,Duration,End
205,ObstructiveApnea,17539.5,21.0,17560.5


In [165]:
breath_df.loc[breath_df['Type'] == 'MixedApnea'].loc[breath_df['Duration'] == 17.5]

,Type,Start,Duration,End
192,MixedApnea,16848.5,17.5,16866.0


In [167]:
print(9104 / 3600, 17539.5 / 3600, 16848.5 / 3600)

2.528888888888889 4.872083333333333 4.680138888888889
